In [2]:
import numpy as np
import pandas as pd
import json
import os
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import roc_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from scipy.stats import randint
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from scipy.stats import randint
import scipy.stats

In [3]:
with open('Datasets/train.json', 'r') as file:
        train_data = json.load(file)
pd.DataFrame(train_data)

,audio_embedding,is_turkey,vid_id,end_time_seconds_youtube_clip,start_time_seconds_youtube_clip
0,"[[172, 34, 216, 110, 208, 46, 95, 66, 161, 125...",0,kDCk3hLIVXo,70,60
1,"[[169, 20, 165, 102, 205, 62, 110, 103, 211, 1...",1,DPcGzqHoo7Y,40,30
2,"[[148, 8, 138, 60, 237, 48, 121, 108, 145, 177...",1,7yM63MTHh5k,240,230
3,"[[151, 0, 162, 88, 171, 71, 47, 90, 179, 190, ...",1,luG3RmUAxxM,520,510
4,"[[162, 17, 187, 111, 211, 105, 92, 67, 203, 15...",0,PIm3cjxTpOk,10,0
...,...,...,...,...,...
1190,"[[90, 105, 224, 173, 117, 88, 119, 2, 219, 59,...",0,AKIWjFmAMt8,30,20
1191,"[[132, 154, 242, 164, 181, 58, 154, 63, 163, 4...",0,QTRCw0xn6uc,40,30
1192,"[[148, 8, 171, 123, 241, 91, 126, 154, 96, 246...",1,DbJJoQfo3kc,40,30
1193,"[[166, 13, 201, 135, 190, 93, 71, 30, 139, 151...",0,opqJ1SZmyGk,200,190


In [4]:
with open('Datasets/test.json', 'r') as file:
        test_data = json.load(file)

pd.DataFrame(test_data)

,audio_embedding,vid_id,end_time_seconds_youtube_clip,start_time_seconds_youtube_clip
0,"[[177, 20, 226, 132, 198, 81, 111, 59, 132, 18...",pyKh38FXD3E,10,0
1,"[[169, 21, 204, 161, 195, 72, 60, 39, 152, 184...",THhP1idrWXA,40,30
2,"[[165, 13, 198, 141, 199, 81, 173, 54, 119, 11...",jsw3T6GY2Nw,40,30
3,"[[167, 18, 188, 159, 198, 63, 156, 36, 179, 22...",nFkXTMHcjMU,24,14
4,"[[178, 32, 181, 100, 198, 46, 82, 83, 136, 227...",Au8g9kAlrLQ,40,30
...,...,...,...,...
1191,"[[153, 26, 118, 69, 255, 87, 153, 59, 125, 204...",lQOQR6HCGyE,11,1
1192,"[[159, 8, 163, 128, 172, 64, 36, 56, 110, 236,...",CqhPMjumTOA,40,30
1193,"[[78, 56, 132, 20, 92, 120, 6, 133, 191, 156, ...",Ccn2LQJc2MI,40,30
1194,"[[81, 115, 208, 104, 236, 91, 95, 71, 161, 152...",f6OhmrIdB-g,40,30


In [5]:
train_data = pd.DataFrame(train_data)

In [6]:
test_data = pd.DataFrame(test_data)

### Bước 1: Tìm tham số tốt nhất cho mô hình SVC bằng phương pháp GridSearchCV 

In [7]:
# # SVM & grid searchCV


# import numpy as np
# import pandas as pd
# from sklearn.svm import SVC
# from sklearn.model_selection import train_test_split, GridSearchCV
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import (
#     roc_auc_score, precision_score,
#     recall_score, f1_score, accuracy_score
# )

# # Chuẩn bị dữ liệu
# train_X = np.stack(train_data['audio_embedding'].apply(lambda x: np.mean(x, axis=0)))
# train_Y = train_data['is_turkey'].values

# valid_idx = test_data['audio_embedding'].apply(lambda x: isinstance(x, list) and len(x) > 0)
# test_X = np.stack(test_data.loc[valid_idx, 'audio_embedding'].apply(lambda x: np.mean(x, axis=0)))

# # Chuẩn hóa
# scaler = StandardScaler()
# Z = scaler.fit_transform(train_X)
# test_Z = scaler.transform(test_X)

# # Chia train/val
# train_Z, val_Z, train_Y, val_Y = train_test_split(Z, train_Y, test_size=0.5, random_state=42)

# # Cấu hình tham số cần tìm
# param_grid = {
#     'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],  # từ rất nhỏ đến rất lớn
#     'gamma': ['scale', 'auto', 1e-4, 1e-3, 1e-2, 0.1, 1, 10],  # từ rất nhỏ đến rất lớn
#     'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],  # tất cả kernel hỗ trợ
#     'degree': [2, 3, 4, 5],  # chỉ áp dụng với 'poly' kernel
#     'class_weight': [None, 'balanced'],  # không áp dụng hoặc cân bằng lớp
#     'shrinking': [True, False],  # bật/tắt thuật toán shrinking heuristic
#     'probability': [True]  # bật tính năng dự đoán xác suất (bắt buộc nếu bạn muốn dùng predict_proba)
# }

# # Khởi tạo GridSearchCV
# grid_search = GridSearchCV(
#     estimator=SVC(probability=True, random_state=42),
#     param_grid=param_grid,
#     scoring='roc_auc', 
#     cv=5,              
#     n_jobs=-1,
#     verbose=2
# )

# # Huấn luyện
# grid_search.fit(train_Z, train_Y)

# # Mô hình tốt nhất
# best_model = grid_search.best_estimator_
# print("Best parameters:", grid_search.best_params_)

# # Dự đoán trên val
# y_pred_prob = best_model.predict_proba(val_Z)[:, 1]
# y_pred = best_model.predict(val_Z)

# # Đánh giá
# print("AUC Score   :", roc_auc_score(val_Y, y_pred_prob))
# print("Accuracy    :", accuracy_score(val_Y, y_pred))
# print("Precision   :", precision_score(val_Y, y_pred))
# print("Recall      :", recall_score(val_Y, y_pred))
# print("F1 Score    :", f1_score(val_Y, y_pred))

# # Dự đoán trên test
# test_pred_prob = best_model.predict_proba(test_Z)[:, 1]
# test_data['is_turkey'] = -1.0
# test_data.loc[valid_idx, 'is_turkey'] = test_pred_prob

# # Xuất kết quả
# test_data.loc[valid_idx, ['vid_id', 'is_turkey']].to_csv('result.csv', index=False)

#### **Kết quả:**
- Từ GridSearchCV và đánh giá bằng kết quả submission thực tế. 
- Ta rút ra model đạt score cao nhất với bộ tham số: 
    - 'class_weight': None 
    - probability = True 
    - Những tham số khác dùng mặc định của model 

### Bước 2: So sánh hiệu quả khi áp dụng thêm 1 số phương pháp khác: kfolds, SMOTE,....
- Kết quả kfolds cho kết quả tốt nhất 

### Bước 3: Tìm tham số random_state và n_splits cho kết quả tốt nhất 
- Sử dụng joblib chạy song song để tối ưu vòng lặp for
- Dùng 2 vòng lặp để vét cạn, tìm ra với mỗi bộ tham số (random_state, n_splits) thì 5 chỉ số AUC, Precision, F1, Recall, Accuracy sẽ đạt bao nhiêu

In [8]:
# # SVM k folds tìm tham số tối ưu nhất 
# import numpy as np
# import pandas as pd
# from sklearn.svm import SVC
# from sklearn.model_selection import StratifiedKFold
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import (
#     roc_auc_score, precision_score,
#     recall_score, f1_score, accuracy_score
# )
# from joblib import Parallel, delayed
# # Chuẩn bị dữ liệu từ embedding
# train_X = np.stack(train_data['audio_embedding'].apply(lambda x: np.mean(x, axis=0)))
# train_Y = train_data['is_turkey'].values

# valid_idx = test_data['audio_embedding'].apply(lambda x: isinstance(x, list) and len(x) > 0)
# test_X = np.stack(test_data.loc[valid_idx, 'audio_embedding'].apply(lambda x: np.mean(x, axis=0)))

# # Chuẩn hóa dữ liệu
# scaler = StandardScaler()
# Z = scaler.fit_transform(train_X)
# test_Z = scaler.transform(test_X)



# auc_list, acc_list, prec_list, recall_list, f1_list = [], [], [], [], []
# models = []

# def evaluate_kfold(random_state_kf, n_splits):
#     kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state_kf)

#     auc_list, acc_list, prec_list, recall_list, f1_list = [], [], [], [], []

#     for train_idx, val_idx in kf.split(Z, train_Y):
#         X_train, X_val = Z[train_idx], Z[val_idx]
#         y_train, y_val = train_Y[train_idx], train_Y[val_idx]

#         model = SVC(
#                 probability=True,
#                 random_state= 42 
#             )
#         model.fit(X_train, y_train)

#         y_pred_prob = model.predict_proba(X_val)[:, 1]
#         y_pred = model.predict(X_val)

#         auc_list.append(roc_auc_score(y_val, y_pred_prob))
#         acc_list.append(accuracy_score(y_val, y_pred))
#         prec_list.append(precision_score(y_val, y_pred, zero_division=0))
#         recall_list.append(recall_score(y_val, y_pred))
#         f1_list.append(f1_score(y_val, y_pred))

#     return {
#             "kf_random_state": random_state_kf,
#             "n_splits": n_splits,
#             "AUC": np.mean(auc_list),
#             "Accuracy": np.mean(acc_list),
#             "Precision": np.mean(prec_list),
#             "Recall": np.mean(recall_list),
#             "F1": np.mean(f1_list)
#         }

# # Tạo các tổ hợp tham số cần đánh giá
# random_state_kf_list = range(1, 100)       # có thể tăng
# n_splits_list = range(21,40)               # tránh split=2 gây lỗi

# param_combinations = [
#     (i, j)
#     for i in random_state_kf_list
#     for j in n_splits_list
# ]

# # Chạy song song
# results = Parallel(n_jobs=-1, verbose=10)(
#     delayed(evaluate_kfold)(i, j) for i, j in param_combinations
# )

# # Chuyển kết quả sang DataFrame
# df_results = pd.DataFrame(results)
# df_results.to_csv("svm_kfold_metrics.csv", index=False)

### Bước 4: Sử dụng bộ tham số mình chọn 
- Chạy model SVC kfolds với bộ tham số tốt nhất, sau đó submit "file result.csv" để xem score đạt được
- Lặp đi lặp lại nhiều lần, bộ tham số đạt score submission cao nhất chính là: 
    - Model: 
        - probability = True 
        - n_splits = 4
        - random_state = 20

In [9]:
# SVM  k - folds - tham số tối ưu nhất 
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    roc_auc_score, precision_score,
    recall_score, f1_score, accuracy_score
)
from joblib import Parallel, delayed
# Chuẩn bị dữ liệu từ embedding
train_X = np.stack(train_data['audio_embedding'].apply(lambda x: np.mean(x, axis=0)))
train_Y = train_data['is_turkey'].values

valid_idx = test_data['audio_embedding'].apply(lambda x: isinstance(x, list) and len(x) > 0)
test_X = np.stack(test_data.loc[valid_idx, 'audio_embedding'].apply(lambda x: np.mean(x, axis=0)))

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
Z = scaler.fit_transform(train_X)
test_Z = scaler.transform(test_X)
# K-Fold Cross Validation
kf = StratifiedKFold(n_splits=4, shuffle=True, random_state=20)

auc_list, acc_list, prec_list, recall_list, f1_list = [], [], [], [], []
models = []

for fold, (train_idx, val_idx) in enumerate(kf.split(Z, train_Y)):
    X_train, X_val = Z[train_idx], Z[val_idx]
    y_train, y_val = train_Y[train_idx], train_Y[val_idx]

    model = SVC(
        probability=True,
        random_state=42
    )
    
    model.fit(X_train, y_train)
    models.append(model)

    y_pred_prob = model.predict_proba(X_val)[:, 1]
    y_pred = model.predict(X_val)

    auc_list.append(roc_auc_score(y_val, y_pred_prob))
    acc_list.append(accuracy_score(y_val, y_pred))
    prec_list.append(precision_score(y_val, y_pred))
    recall_list.append(recall_score(y_val, y_pred))
    f1_list.append(f1_score(y_val, y_pred))

    print(f"Fold {fold+1} — AUC: {auc_list[-1]:.4f}, F1: {f1_list[-1]:.4f}")

# Trung bình các chỉ số đánh giá
print("\n=== Trung bình K-Fold ===")
print("AUC Score   :", np.mean(auc_list))
print("Accuracy    :", np.mean(acc_list))
print("Precision   :", np.mean(prec_list))
print("Recall      :", np.mean(recall_list))
print("F1 Score    :", np.mean(f1_list))

# Dự đoán trung bình trên tập test
test_pred_probs = np.mean([model.predict_proba(test_Z)[:, 1] for model in models], axis=0)
test_data['is_turkey'] = -1.0
test_data.loc[valid_idx, 'is_turkey'] = test_pred_probs

# Xuất kết quả
test_data.loc[valid_idx, ['vid_id', 'is_turkey']].to_csv('result.csv', index=False)

Fold 1 — AUC: 0.9895, F1: 0.9461
Fold 2 — AUC: 0.9879, F1: 0.9478
Fold 3 — AUC: 0.9836, F1: 0.9160
Fold 4 — AUC: 0.9898, F1: 0.9412

=== Trung bình K-Fold ===
AUC Score   : 0.9876916657579392
Accuracy    : 0.9497934950955085
Precision   : 0.9539882399397557
Recall      : 0.9225976276156205
F1 Score    : 0.9377480282719497
